Imports

In [20]:
import csv
import pandas as pd
from datetime import datetime
from datetime import timedelta
import os
#from subprocess import call
import subprocess

Parameters

In [21]:
# source filepath
sourceFilepath = '/home/julio/strides/data/activities_until_19_02_06/'
#sourceFilepath = '/home/julio/strides/data/fit_raw/'

Functions

In [22]:
#convert .FIT to .csv with the help of FIT_sdk
def convertFITToCsv(sourceFilepath, file):
    source_file = sourceFilepath+file+".FIT"
    target_file = "/home/julio/strides/data/fit2csv/"+file+".csv"
    #call(["java", "-jar", "/home/julio/strides/fit_sdk/java/FitCSVTool.jar", "-b", source_file, target_file])
    subprocess.run(["java", "-jar", "/home/julio/strides/fit_sdk/java/FitCSVTool.jar", "-b", source_file, target_file], stdout=subprocess.PIPE)

In [23]:
def secondsToUTC (dataset):
    #1989-12-31 00:00:00
    zero = datetime(1989,12,31,0,0,0)
    #print (datetime.strftime(zero,"%b %d %Y %H:%M:%S"))
    new_dataset = []
    for d in dataset:
        n = str(zero + timedelta(seconds=d))
        new_dataset.append(n)
    return new_dataset

In [24]:
def prepareCSV (filepath, file, filetype):
    print(file)
    #Read csv
    df = pd.read_csv(filepath+file+filetype)
    
    #filter for local number == 7 and type == Date
    df_filtered = df.loc[df['Local Number'] == 7]
    df_filtered = df_filtered.loc[df_filtered['Type'] == 'Data']
    
    #convert position and time
    df_filtered['Value 2'] = df_filtered['Value 2']* 180/(2**31)
    df_filtered['Units 2'] = "degrees"

    df_filtered['Value 3'] = df_filtered['Value 3']* 180/(2**31)
    df_filtered['Units 3'] = "degrees"

    df_filtered['Value 1'] = secondsToUTC(df_filtered['Value 1'])
    df_filtered['Units 1'] = "UTC"
    
    # drop unnecessary data
    toBeDropped = []
    counter = 10
    field_str = 'Field '
    value_str = 'Value '
    units_str = 'Units '

    upperBound = (len(df.columns) - 31)/3 + 10
    while counter < upperBound:
        toBeDropped.append(field_str+str(counter))
        toBeDropped.append(value_str+str(counter))
        toBeDropped.append(units_str+str(counter))
        counter = counter + 1

    toBeDropped.append("Type")
    toBeDropped.append("Local Number")
    toBeDropped.append("Message")
    toBeDropped.append("Unnamed: "+str(len(df.columns)-1))
    
    df_filtered = df_filtered.drop(columns=toBeDropped)
    
    # write csv
    df_filtered.to_csv(filepath+file+'_prepared'+filetype)

In [33]:
#Get file names in directory
def getFiles(sourceFilepath, filetype):
    files = []  
    for file in os.listdir(sourceFilepath):
        if file.endswith(filetype) and not file.endswith("prepared"+filetype):
            files.append(file.split('.')[0])
    return files

Convert FIT to csv

In [34]:
files = getFiles(sourceFilepath=sourceFilepath, filetype='.FIT')
print (files)
for file in files:
    convertFITToCsv(sourceFilepath=sourceFilepath, file=file)

['395292577', '1391805895', '1623815420', '487002461', '777445698', '522710125', '1519068264', '2218383659', '286398336', '314699243', '2111948713', '572833534', '744953305', '392780637', '1906938350', '726227115', '362495582', '206292230', '2260608410', '1381684376', '294663272', '1333896356', '771052332', '1777591783', '1646282581', '310197205', '438111608', '257872553', '335101812', '1367501623', '281805850', '1664561448', '200088451', '1725452373', '2111950691', '362495564', '337478560', '699011415', '439547876', '665702120', '1368904075', '210396303', '249484270', '418482879', '220024739', '273024824', '337478458', '1604248070', '202369852', '1776149366', '1158041812', '700666368', '1787178780', '392780710', '1742796534', '368892701', '200734680', '675288520', '1449998509', '1689246197', '2198144256', '1187716998', '712554760', '1639258689', '623858556', '305221565', '239055500', '984763343', '1780602929', '1025710236', '685916036', '2215264916', '335101786', '309882242', '1573004

Prepare csv files

In [37]:
files = getFiles(sourceFilepath='/home/julio/strides/data/fit2csv/', filetype='.csv')
print (files)
for file in files:
    prepareCSV (filepath='/home/julio/strides/data/fit2csv/', file=file, filetype='.csv')

['348235268', '744890190', '687309101', '1374520546', '2113236689', '421499913', '1822520231', '476772035', '908039774', '307444024', '712554760', '348235293', '1184730049', '1773626685', '200734685', '222358690', '1018496029', '1434437828', '284433799', '767992238', '356894725', '2241591040', '666171237', '2111948713', '1655559538', '307443796', '382651306', '2111943934', '2228050321', '302297807', '368881548', '1180153693', '204528494', '1644288277', '579009887', '226382750', '2170034983', '965163324', '1557917860', '260622623', '666171307', '385720690', '623859572', '288624127', '1281975128', '657659493', '1417423910', '372834140', '623859596', '723844886', '1730787213', '664291299', '1421995341', '2118971771', '284433787', '984763318', '1438070107', '481957999', '298184960', '1423547898', '806444263', '1238859934', '2234667042', '1392466718', '927360743', '447662292', '1682139357', '404227161', '1793679321', '305221920', '343718304', '1665934716', '665702094', '318950801', '1806113

744890190


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (32,35,41,44,83,111,114,117) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


687309101
1374520546
2113236689
421499913
1822520231
476772035
908039774
307444024
712554760
348235293
1184730049
1773626685
200734685
222358690
1018496029
1434437828
284433799
767992238
356894725
2241591040
666171237
2111948713
1655559538
307443796
382651306
2111943934
2228050321
302297807
368881548
1180153693
204528494
1644288277
579009887
226382750
2170034983
965163324
1557917860
260622623
666171307
385720690
623859572
288624127
1281975128
657659493
1417423910
372834140
623859596
723844886
1730787213
664291299
1421995341
2118971771


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (32,35,41,44,47,48,50,51,53,54,56,57,59,60,62,63,66,69,72,75,77,78,80,81,83,84,87,90,93,96,99,102,105,108,111,114,117) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


...

1760378954
1906938350
1376059752
541950078
587344224
1798231978
393664064
1639261077
217953876
1323239603
1187716998
928437785
1493085935
864403638
479200288
817276853
1575970955
900609081
1578774551
1632527058
307443753
2035645647
1323238532
326844730
1519068264
477185683
723844851
1452241799
1023244284
243116295
1524209177
2260608410
368892744
1604248070
1621193718
1054556662
1990671371
256740194
382651290
298184980
415068991
991288621
382651458
675288520
277379084
385720673
699011415
1452241788
1526401356
1543244499
1529747207
604850092
1393796126
1662489829
757698603
1393796947
506649703
400295436
356895102
1780603069


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (33,35,36,39,41,42,44,45,47,48,50,51,53,54,56,57,59,60,62,63,66,69,71,72,74,75,77,78,80,81,83,84,86,87,89,90,93,96,99,102,105,108,111,114,117) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


486971983
243116301
2123148749
613631202
522710125
680926498
2093961231
1827396150
348235242
271638907
1501235861
476772020
222358458
377415853
309882242
1333889695
1098212484
1514250978
226382711
2268891470
660697732
478171051
337478602
368881528
687949296
203996615
288481383
2188857330
483789667
1419822034
1973240162
2041211598
278482332
355231700
368881569
1010196989
1477520598
251272045
273024824
331364462
680926525
1443078616
1979016546
657659469
1675992597
210396303
1915534559
499569749
404226453
305221548
476772015
777445698
1066372552
377433385
1108298889
1692579247
267112987
535842693
1662489771
623859490
847475289
314699243
1618098436
2170031829
398965560
2127938269
1664561515
206810031
2250579369
1434437830
210006761
1789315314
496191522
995207294
1430314839
1787885236
1625985312
2259156621
2093962504
1073818614
331363803
1278408661
1556341221
1159570262
377433390
1467091039
1520749381
1538279859
309882229
245701517
834178826
792456448
1819755731
774246685
340051920
16238154

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (32,35,41,44,71,74,83,111,114,117) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


1025710236
1621466628
744890872
305221590
1344705688
1664561509
255658989
1339959253
465825051
685916036
2041131975
284433772
350250310
1230679071
400295608
244035446
1675992959
1623816893
1393795239
1075439204
596966839
1435717541
259341746
496094685
1557917833
335101786
1623816883
1050305329
2111950691
1141883963
1003302646
1594123733
217954168
509154577
465824946
1650495734
971452886
1072581994
669921300
249781641
623858556
961119572
885246679
331606381
649035551
512582466
1561215247
275908358
1367501623
717164498
2093966836
258292747
1460206837
497464091
2212501098
314699029
392780710
307444112
1932136568
1591248974
1175683783
362529440
1491463714
263751627
2234665306
509824047
249484270
1646282581
1828843466
1156144164
1754024514
310197205
2170033605
377415866
382651317
243116298
280911539
400295555
337478139
1407716586
1078797883
206292667
331364506
252313931
932288704
1323235658
212575192
1795792217
946587363
502095146
203517773
355231716
314699275
518490954
2226627460
898587152